## Dynamic map interface for DFlight API

In this notebook you'll use [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/index.html) to create an interactive map which allows you to:
* define request geometries by drawing them on map
* create and send API requests
* visualize returned results

Please review DFlight documentation before getting started:
* [Overview](https://ljaero.com/solutions/dflight/overview/)
* [Developer Resources](https://ljaero.com/solutions/dflight/dev/)
* [OpenAPI Specification](https://dflight-api.ljaero.com/)

## Enter your API Key
In order to make calls to DFlight API enpoints you need to include your API Key in the ```x-api-key``` header. Replace ```nnnn``` below with your key. If you do not already have a DFlight subscription, click [here](https://ljaero.com/solutions/dflight/subscribe/) for a free trial which will allow you to make 50 API calls.

In [ ]:
mykey = 'nnnn'
headers = {'x-api-key': mykey}

## Requirements

In order to execute this notebook, the following open source packages must be installed in your environment:
* [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation/index.html)
* [pyproj](https://pyproj4.github.io/pyproj/stable/installation.html)
* [requests](https://requests.readthedocs.io/en/latest/user/install/#install)
* [shapely](https://shapely.readthedocs.io/en/stable/installation.html)

In [ ]:
# For sending HTTP requests
import requests

# For creating and interacting with map
from ipyleaflet import Map, GeoJSON, GeoData, WKTLayer, LayerGroup, CircleMarker, basemaps
from ipyleaflet import DrawControl, LayersControl, LegendControl, ScaleControl, ZoomControl, WidgetControl, FullScreenControl
import ipywidgets as widgets

# Helpers for working with shape data
import pyproj
import shapely.geometry as shpgeo
from shapely.ops import transform
from shapely.geometry import shape

## Create interactive map
Create map that you'll use to draw geometries, send API requests, and view returned results. The controls we'll add are:
* Input tools for making polygons, lines, and point/distance circles
* Drop-down to select which endpoint to use
* Full screen view
* Layer selector
* Map scale indicator
* Text display area
* Button to compose and send API request based on inputs
* Button to clear the map and start over

In [ ]:
mapsize = widgets.Layout(width='960px', height='480px')
center = (38.0, -95.0)
leafmap = Map(center = center, zoom = 4.45, layout = mapsize, 
              basemap = basemaps.Esri.WorldTopoMap, scroll_wheel_zoom = True)
leafmap.add_control(ScaleControl(metric = True, imperial = True))
leafmap.add_control(LayersControl())
leafmap.add_control(FullScreenControl())

#### Add DrawControl for creating lines, polygons, point/distance circles then capture areas you've drawn.

When you draw a new polygon or route, or drag a new circle, you need to capture the geometry of the area to use it in the API request. The variables and ```handle_draw``` method below will take care of that.

In [ ]:
# The input_shapes list will hold all geometries you've drawn.
input_shapes = []

# When you drag a circle on the map only the Point coordinates are saved in the draw event's geo_json 'geometry'.
# The radius gets put into the geo_json 'properties' so we need to save those also.
input_properties = []

def make_circle_poly(lon: float, lat: float, dist: float) -> shpgeo.polygon.Polygon:
    """
    This helper method creates a circular polygon from Point + radius
    """
    wgs84 = pyproj.CRS('EPSG:4326')
    utm = pyproj.CRS('EPSG:3857')
    to_utm = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
    to_wgs84 = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
    pt = shpgeo.Point(lon, lat)
    t_pt = transform(to_utm, pt)
    buf = t_pt.buffer(dist)
    return transform(to_wgs84, buf)

def handle_draw(self, action, geo_json):
    """
    Actions taken when you draw a new shape on map. Ipyleaflet will only show the center marker when you drag a circle,
    so in addition to saving the geometry + input_properties we create a circular polygon for your requested area
    and display it on the map.
    """
    if action == 'created':
        geo = geo_json.get('geometry')
        props = geo_json.get('properties')
        input_shapes.append(geo)
        input_properties.append(props)          
        if geo['type'] == 'Point':
            rc = shpgeo.mapping(make_circle_poly(geo['coordinates'][0], geo['coordinates'][1], props['style']['radius']))
            layer = GeoJSON(data = rc,style ={'color': 'black', 'fillColor': 'black', 'opacity':1.0, 'weight': 2, 'dashArray': '8', 'fillOpacity': 0.05})            
            layer.name = "requested area"
            leafmap.add_layer(layer)

draw_control = DrawControl()
draw_control.polyline =  {
    "shapeOptions": {
        "color": "black",
        "weight": 6,
        "opacity": 1.0,
        "weight": 2, 
        "dashArray": "8"
    }
}
draw_control.polygon = {
    "shapeOptions": {
        "opacity": 1.0,
        "fillColor": "black",
        "color": "black",
        "fillOpacity": 0.05,
        "weight": 2, 
        "dashArray": "8"
    },
    "allowIntersection": False
}
draw_control.circle = {
    "shapeOptions": {
        "opacity": 1.0,
        "fillColor": "#efed69",
        "color": "#6bc2e5",
        "fillOpacity": 0.5,
        "fill": True
    }
}
draw_control.circlemarker = {}
draw_control.on_draw(handle_draw)
leafmap.add_control(draw_control)

#### Add drop-down
Add a drop-down selector to choose which endpoint group (information category) you want to retreive. In this notebook we're only goint to include the simplest DFlight API endpoints, i.e. those that:
1. contain just area definition in the request with no additional required elements
2. return a single FeatureCollection with single geometries for each feature.
These are:

|Category    |          | Endpoint Group |
| :- | :- |:- |
|Special Security Areas| |  ```https://dflight-api.ljaero.com/us/v1/ssa/```    |
|Restricted Public Venues| | ```https://dflight-api.ljaero.com/us/v1/venues/``` |
|Surface Obstacles| | ```https://dflight-api.ljaero.com/us/v1/obstacles/```|
|Aerodromes| | ```https://dflight-api.ljaero.com/us/v1/aerodromes/```|
|UAS Operating Areas| | ```https://dflight-api.ljaero.com/us/v1/uoa/```|

In [ ]:
base_url_options = [('Aerodromes','https://dflight-api.ljaero.com/us/v1/aerodromes/'),
                    ('Restricted Public Venues','https://dflight-api.ljaero.com/us/v1/venues/'),
                    ('Special Security Areas','https://dflight-api.ljaero.com/us/v1/ssa/'),
                    ('Surface Obstacles','https://dflight-api.ljaero.com/us/v1/obstacles/'),
                    ('UAS Operating Areas','https://dflight-api.ljaero.com/us/v1/uoa/')]

endpoint_selector = widgets.Dropdown(options = base_url_options, 
                                     value = 'https://dflight-api.ljaero.com/us/v1/aerodromes/',
                                     description='Category:', disabled=False,)
leafmap.add_control(endpoint_selector)

#### Add response message area

Create a text area to display informational and error messages for submitted requests

In [ ]:
output_text = widgets.HTML(value = "", layout = widgets.Layout(width = '95%', disabled = False))
leafmap.add_control(output_text)

#### Add "Send Request" button

Cell below includes:
1. methods to create and send API request based on your inputs
2. method to display returned results on map, or print message if no results found
3. code to create the actual button

In [ ]:
# The responses list will hold all of the API responses
responses = []

def get_url() -> str:
    """
    Create full url based on drop-down selection and map input
    """
    base_url = endpoint_selector.value
    req_type = input_shapes[-1]['type']
    if req_type == 'Polygon':
        return base_url + 'polygon-query'
    if req_type == 'LineString':
        return base_url + 'route-query'
    return base_url + 'distance-query' 

def compose_request(input_area: dict) -> dict:  
    """
    Create request json from map input
    """
    req = {}  
    req_type = input_area['type']    
    if req_type == 'Polygon':        
        req['poly'] = input_area
        return req
    if req_type == 'LineString':
        req['route'] = input_area
        return req    
    req['longitude'] = input_area['coordinates'][0]
    req['latitude'] = input_area['coordinates'][1]
    req['distance'] = input_properties[-1]['style']['radius']
    return req

def send_request():
    """
    Create request json and POST to appropriate url
    """
    return requests.post(get_url(), headers = headers, json = compose_request(input_shapes[-1]))

def get_color(cat: str) -> str:
    """
    Give each category it's own color so that results from multiple can be distinguished when displayed together on map.
    """
    colors = {'Special Security Areas': '#EC610E',
              'Restricted Public Venues': '#70943C',
              'Surface Obstacles': '#FF01F3',
              'Aerodromes': '#00AFAA',
              'UAS Operating Areas': '#3F59A7'}
    return colors[cat]

def request_button_click(a):   
    """
    1. Send request to appropriate DFlight endpoint
    2. Save response in responses list
    3. If response 'found' element has anything in it, display on map. If empty or error, print the response.
       - For endpoints that return Point geometries (aerodromes and obstacles), circle markers are plotted
       - For endpoints that return Polygon geometries (all the others), colored areas are drawn
    """
    response = send_request()
    responses.append(response)    
    if response.status_code == 200:  
        res = response.json()
        found = res['found']  
        features = found['features']    
        cat = endpoint_selector.label          
        if features:               
            layer_group = LayerGroup()
            if cat in ['Aerodromes', 'Surface Obstacles']:
                for i in range (0, len(features)):
                    lon, lat = features[i]['geometry']['coordinates']
                    circle_marker = CircleMarker()
                    circle_marker.location = (lat, lon)
                    circle_marker.radius = 6
                    circle_marker.color = get_color(cat)
                    circle_marker.fill_color = get_color(cat)
                    layer_group.add_layer(circle_marker)
            else:
                sd = {}        
                sd['opacity'] = 1
                sd['fillOpacity'] = 0.5
                sd['color'] = get_color(cat)
                sd['fillColor'] = get_color(cat)
                for i in range (0, len(features)):
                    feat = features[i]
                    layer = GeoJSON(data = feat, 
                                    style = sd,
                                    hover_style = {'color': 'black', 'fillColor': 'white', 'fillOpacity': 0.5})
                    layer_group.add_layer(layer)
            layer_group.name = cat
            leafmap.add_layer(layer_group)              
            output_text.value += '<br>' + f'{len(features)} {cat} found in requested area'
        else:
            # If there were no features found, print message saying so
            output_text.value += '<br>' + f'No {cat} found in requested area'            
                
    else:
        if response.status_code == 422:   
            output_text.value += '<br>' + response.json()['detail'][0]['msg']
        else:            
            output_text.value += '<br>' + response.text

send_button = widgets.Button(description = 'Send Request')  
leafmap.add_control(send_button)
send_button.on_click(request_button_click)

#### Add "Clear All" button

Clicking this button will allow you to "start over". It will remove all inputs and results displayed on the map and also clear the ```responses```, ```input_shapes```, and ```input_properties``` lists.

In [ ]:
def clear_button_click(a):
    """
    - Clear all holder lists    
    - Remove print messages below map
    - Remove all map layers except the basemap
    """
    draw_control.clear()
    output_text.value = ""
    del responses[:]    
    del input_shapes[:]
    del input_properties[:]   
    current_layers = list(leafmap.layers)
    if len(current_layers) > 1:
        for i in range(1, len(current_layers)):
            leafmap.remove_layer(current_layers[i])
        
clear_button = widgets.Button(description = 'Clear All')
leafmap.add_control(clear_button)
clear_button.on_click(clear_button_click)    

#### Display the map with control widgets

Once the cell below is executed you can begin using the map to make requests and view results.

In [ ]:
items = [endpoint_selector, send_button, clear_button]
buttons = [send_button, clear_button]
display(widgets.Box(items))
display(leafmap)
display(output_text)